https://huggingface.co/docs/transformers/en/training

In [1]:
!pip install -qq adapters datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 60.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("hrithikpiyush/acl-arc")
dataset = dataset.rename_column(original_column_name="intent", new_column_name="labels")

train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1688 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/114 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/139 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allenai/cs_roberta_base")

def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

tokenized_train_data = train_data.map(tokenize_function, batched=True, batch_size=len(train_data))
tokenized_val_data = val_data.map(tokenize_function, batched=True, batch_size=len(val_data))
tokenized_test_data = test_data.map(tokenize_function, batched=True, batch_size=len(test_data))

tokenizer_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Map:   0%|          | 0/1688 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

Map:   0%|          | 0/139 [00:00<?, ? examples/s]

In [4]:
tokenized_train_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [5]:
from transformers import AutoModelForSequenceClassification

num_labels = len(set(train_data['labels']))
model = AutoModelForSequenceClassification.from_pretrained("allenai/cs_roberta_base", num_labels=num_labels)

pytorch_model.bin:   0%|          | 0.00/656M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allenai/cs_roberta_base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import numpy as np
import evaluate

# https://huggingface.co/spaces/evaluate-metric/f1/blob/main/f1.py
# Example 5-A multi-label example
metric = evaluate.load("f1", "multilabel")

In [7]:
# https://medium.com/@rakeshrajpurohit/customized-evaluation-metrics-with-hugging-face-trainer-3ff00d936f99

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    precision_macro = precision_score(labels, preds, average='macro')
    precision_weighted = precision_score(labels, preds, average='weighted')
    recall_macro = recall_score(labels, preds, average='macro')
    recall_weighted = recall_score(labels, preds, average='weighted')
    f1_macro = f1_score(labels, preds, average='macro')
    f1_weighted = f1_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'precision_weighted': precision_weighted,
        'recall_macro': recall_macro,
        'recall_weighted': recall_weighted,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=5,  # <- change from 6 to 5
    per_device_train_batch_size=32,  # <- default is 8 idk what is better
    per_device_eval_batch_size=32,  # <- default is 8
    auto_find_batch_size=True,  # automates lowering the batch size to resolve out-of-memory errors
    logging_steps=100,
    output_dir="./training_output_4",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    evaluation_strategy="epoch"
)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Precision Weighted,Recall Macro,Recall Weighted,F1 Macro,F1 Weighted
1,1.182400,1.000762,0.684211,0.342460,0.622953,0.337907,0.684211,0.295864,0.600639
2,0.812700,0.773450,0.728070,0.718349,0.759732,0.580690,0.728070,0.617615,0.731186
3,0.484600,0.645078,0.798246,0.795510,0.808998,0.701258,0.798246,0.727937,0.790703
4,0.296700,0.856838,0.798246,0.709052,0.796821,0.731766,0.798246,0.718149,0.795866
5,0.166400,0.827093,0.807018,0.693064,0.813177,0.734591,0.807018,0.708979,0.808219


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=530, training_loss=0.56303204410481, metrics={'train_runtime': 769.1381, 'train_samples_per_second': 10.973, 'train_steps_per_second': 0.689, 'total_flos': 2220737060782080.0, 'train_loss': 0.56303204410481, 'epoch': 5.0})

In [11]:
trainer.evaluate()

{'eval_loss': 0.8270934820175171,
 'eval_accuracy': 0.8070175438596491,
 'eval_precision_macro': 0.6930640559950905,
 'eval_precision_weighted': 0.8131770125418039,
 'eval_recall_macro': 0.7345910680656443,
 'eval_recall_weighted': 0.8070175438596491,
 'eval_f1_macro': 0.7089793281653747,
 'eval_f1_weighted': 0.8082188675823927,
 'eval_runtime': 3.4432,
 'eval_samples_per_second': 33.109,
 'eval_steps_per_second': 1.162,
 'epoch': 5.0}

In [12]:
trainer.predict(tokenized_test_data)

PredictionOutput(predictions=array([[-0.44019258, -2.2277262 ,  6.1795907 , -1.3346813 , -0.4647056 ,
        -1.7037069 ],
       [ 6.0861073 , -0.8950125 , -2.3834383 , -1.7516224 , -0.6446087 ,
        -1.8306626 ],
       [ 0.07014158, -1.471502  , -1.4295163 , -0.5139357 ,  5.1659665 ,
        -0.30297232],
       [ 5.854413  , -1.0642124 , -0.8034522 , -1.4342566 , -1.8516911 ,
        -2.4198172 ],
       [ 0.06584813, -1.1636444 , -1.629442  , -0.78076154, -1.0755284 ,
         4.9534674 ],
       [ 5.9507065 , -0.45928866, -2.6250985 , -1.573998  , -0.7292844 ,
        -1.9618965 ],
       [ 4.3040805 , -0.8489601 , -1.8826861 , -2.2514505 , -0.03964057,
        -0.22147518],
       [ 5.9608517 , -0.41592136, -2.2396855 , -1.5996771 , -1.4618995 ,
        -1.803286  ],
       [ 6.001134  , -0.58728296, -1.9982529 , -1.6634719 , -1.6396286 ,
        -1.8295985 ],
       [ 5.9019036 , -0.2734768 , -2.0971622 , -1.4588464 , -1.7214115 ,
        -2.015171  ],
       [ 6.0248084 , 

In [ ]:
from google.colab import runtime
runtime.unassign()